In [1]:
# Importing necessary modules
import numpy as np#
import matplotlib.pyplot as plt
from astropy.time import Time
from astropy import units as u
from datetime import datetime, timedelta
from astropy.coordinates import solar_system_ephemeris, EarthLocation, get_sun, get_body

### Reference Times

The time at which an astronomical event (a supernova, an eclipse, a gamma-ray burst) usually occurs is typically given in Julian days (JD). This is the time which has lapsed, in days, since noon at Greenwich on January 1, 4713 BC. There are variants such as the Modified Julian Date (MJD), which is the time in days since midnight, November 17 1858. MJD is typically used instead of JD simply because it removes a very large constant. The below code shows todays JD and MJD (as computed when I last evaluated this cell).

In [2]:
ut = Time(datetime.utcnow(), scale='utc')
#ut = Time(datetime(2024,3,21,0,0,0,0), scale='utc')

print(f"The current time is: {ut.isot}")
print(f"The current time in JD is: {ut.jd}")
print(f"The current time in MJD is: {ut.mjd}")

The current time is: 2025-01-07T13:29:42.375
The current time in JD is: 2460683.0622960087
The current time in MJD is: 60682.56229600859


In [4]:
ut = Time(datetime(2024,3,21,0,0,0,0), scale='utc')

print(f"The current time is: {ut.isot}")
print(f"The current time in JD is: {ut.jd}")
print(f"The current time in MJD is: {ut.mjd}")

The current time is: 2024-03-21T00:00:00.000
The current time in JD is: 2460390.5
The current time in MJD is: 60390.0
